In [41]:
import os
import pandas as pd
from kernels.wrapper import MODELS, KernelModelWrapper
from dataset.ipc2023_learning_domain_info import IPC2023_LEARNING_DOMAINS, get_test_ipc2023_learning_instance_files, get_best_bound
from itertools import product
from IPython.display import display, HTML
from util.scrape_log import scrape_search_log
from util.visualise import *

In [42]:
pd.options.display.float_format = "{:.1f}".format

In [43]:
_LOG_DIR = "logs/test_ipc2023"
_DIFFICULTIES = ["easy", "medium", "hard"]
_ITERATIONS = [str(i) for i in [1]]
_MODELS = ["linear-regression", "linear-svr", "ridge", "lasso", "rbf-svr", "quadratic-svr", "cubic-svr"]
CONFIGS = list(product(_MODELS, _ITERATIONS))

In [44]:
def get_data(domain):
  data = {
  }

  test_files = get_test_ipc2023_learning_instance_files()

  for config in CONFIGS:
    d = {
      "solved": [],
      "expanded": [],
      "cost": [],
      "first_h": [],
      "quality": [],
      "c_opt": [],
      "difficulty": [],
    }
    model, iterations = config

    for difficulty in _DIFFICULTIES:
      for df, pf in test_files[domain]:
        if not difficulty in pf:  # TODO generalise
          continue
        pf_name = os.path.basename(pf).replace(".pddl", "")
        log_file = f"{_LOG_DIR}/{domain}_{difficulty}_{pf_name}_trained_models_kernel-{model}_llg_ipc2023-learning-{domain}_wl_{iterations}.joblib.log"
        
        if not os.path.exists(log_file):
          continue

        stats = scrape_search_log(log_file)
        for k in d:
          if k in stats:
            d[k].append(stats[k])
        
        c = stats["cost"]
        c_opt = get_best_bound(domain, difficulty, pf_name)
        quality = c_opt / c
        quality = max(quality, 0)
        d["quality"].append(quality)
        d["c_opt"].append(c_opt)
        d["difficulty"].append(difficulty)
        
    data[config] = d

  return data

In [45]:
def get_metrics_df(data):
  d = {
    "config": [],
    "quality": [],
    "solved": [],
    "tried": [],
    "e_quality": [],
    "e_solved": [],
    "e_tried": [],
    "m_quality": [],
    "m_solved": [],
    "m_tried": [],
    "h_quality": [],
    "h_solved": [],
    "h_tried": [],
  }
  for config in CONFIGS:
    d["config"].append("_".join(config))
    d["quality"].append(sum(data[config]["quality"]))
    d["solved"].append(sum(data[config]["solved"]))
    d["tried"].append(len(data[config]["solved"]))
    for difficulty in _DIFFICULTIES:
      quality = 0
      solved = 0
      tried = 0
      for i, prob_solved in enumerate(data[config]["solved"]):
        if data[config]["difficulty"][i] == difficulty:
          quality += data[config]["quality"][i]
          solved += prob_solved
          tried += 1
      d[f"{difficulty[0]}_quality"].append(quality)
      d[f"{difficulty[0]}_solved"].append(solved)
      d[f"{difficulty[0]}_tried"].append(tried)
  return pd.DataFrame(d)

In [46]:
def get_table_df():
  d = {
    "domain": [],
  }
  for config in CONFIGS:
    d['_'.join(config)] = []

  sum_scores = {config: 0 for config in CONFIGS}

  for domain in IPC2023_LEARNING_DOMAINS:
    data = get_data(domain)
    d["domain"].append(domain)
    for config in CONFIGS:
      score = sum(data[config]["quality"])
      d['_'.join(config)].append(score)
      sum_scores[config] += score
  

  d["domain"].append("sum")
  for config in CONFIGS:
    d['_'.join(config)].append(sum_scores[config])

  return pd.DataFrame(d)

In [47]:
display(get_table_df())

,domain,linear-regression_1,linear-svr_1,ridge_1,lasso_1,rbf-svr_1,quadratic-svr_1,cubic-svr_1
0,blocksworld,5.3,17.8,16.8,22.2,14.5,12.1,8.9
1,childsnack,8.5,10.0,10.0,8.0,9.0,9.0,9.0
2,ferry,24.4,78.0,77.1,77.1,30.5,30.5,30.5
3,floortile,1.0,0.9,1.8,2.0,0.0,0.0,0.0
4,miconic,29.2,28.8,28.9,28.7,28.7,28.7,28.6
5,rovers,9.3,17.3,23.6,18.4,14.3,15.3,14.5
6,satellite,10.0,4.1,4.8,10.2,10.8,32.4,15.8
7,sokoban,21.8,34.8,36.6,36.6,27.5,26.4,26.3
8,spanner,25.9,25.9,30.0,30.0,25.9,25.9,26.0
9,transport,7.2,25.9,33.1,33.1,25.1,23.6,26.1


In [48]:
for domain in IPC2023_LEARNING_DOMAINS:
  print(domain)
  data = get_data(domain)
  display(get_metrics_df(data))

blocksworld


,config,quality,solved,tried,e_quality,e_solved,e_tried,m_quality,m_solved,m_tried,h_quality,h_solved,h_tried
0,linear-regression_1,5.3,6,59,5.3,6,30,0.0,0,29,0,0,0
1,linear-svr_1,17.8,27,60,17.8,27,30,0.0,0,30,0,0,0
2,ridge_1,16.8,29,60,16.3,28,30,0.5,1,30,0,0,0
3,lasso_1,22.2,36,60,19.0,30,30,3.3,6,30,0,0,0
4,rbf-svr_1,14.5,22,22,14.5,22,22,0.0,0,0,0,0,0
5,quadratic-svr_1,12.1,19,19,12.1,19,19,0.0,0,0,0,0,0
6,cubic-svr_1,8.9,14,14,8.9,14,14,0.0,0,0,0,0,0


childsnack


,config,quality,solved,tried,e_quality,e_solved,e_tried,m_quality,m_solved,m_tried,h_quality,h_solved,h_tried
0,linear-regression_1,8.5,9,60,8.5,9,30,0,0,30,0,0,0
1,linear-svr_1,10.0,10,60,10.0,10,30,0,0,30,0,0,0
2,ridge_1,10.0,10,60,10.0,10,30,0,0,30,0,0,0
3,lasso_1,8.0,8,60,8.0,8,30,0,0,30,0,0,0
4,rbf-svr_1,9.0,9,9,9.0,9,9,0,0,0,0,0,0
5,quadratic-svr_1,9.0,9,9,9.0,9,9,0,0,0,0,0,0
6,cubic-svr_1,9.0,9,9,9.0,9,9,0,0,0,0,0,0


ferry


...

,config,quality,solved,tried,e_quality,e_solved,e_tried,m_quality,m_solved,m_tried,h_quality,h_solved,h_tried
0,linear-regression_1,21.8,23,90,21.8,23,30,0.0,0,30,0,0,30
1,linear-svr_1,34.8,36,90,26.9,30,30,7.9,6,30,0,0,30
2,ridge_1,36.6,37,90,27.6,30,30,9.0,7,30,0,0,30
3,lasso_1,36.6,37,90,27.6,30,30,9.0,7,30,0,0,30
4,rbf-svr_1,27.5,30,30,27.5,30,30,0.0,0,0,0,0,0
5,quadratic-svr_1,26.4,29,29,26.4,29,29,0.0,0,0,0,0,0
6,cubic-svr_1,26.3,29,29,26.3,29,29,0.0,0,0,0,0,0


spanner


,config,quality,solved,tried,e_quality,e_solved,e_tried,m_quality,m_solved,m_tried,h_quality,h_solved,h_tried
0,linear-regression_1,25.9,30,60,25.9,30,30,0,0,30,0,0,0
1,linear-svr_1,25.9,30,60,25.9,30,30,0,0,30,0,0,0
2,ridge_1,30.0,30,60,30.0,30,30,0,0,30,0,0,0
3,lasso_1,30.0,30,60,30.0,30,30,0,0,30,0,0,0
4,rbf-svr_1,25.9,30,30,25.9,30,30,0,0,0,0,0,0
5,quadratic-svr_1,25.9,30,30,25.9,30,30,0,0,0,0,0,0
6,cubic-svr_1,26.0,30,30,26.0,30,30,0,0,0,0,0,0


transport


,config,quality,solved,tried,e_quality,e_solved,e_tried,m_quality,m_solved,m_tried,h_quality,h_solved,h_tried
0,linear-regression_1,7.2,8,60,7.2,8,30,0.0,0,30,0,0,0
1,linear-svr_1,25.9,33,60,22.8,29,30,3.1,4,30,0,0,0
2,ridge_1,33.1,30,60,33.1,30,30,0.0,0,30,0,0,0
3,lasso_1,33.1,30,60,33.1,30,30,0.0,0,30,0,0,0
4,rbf-svr_1,25.1,24,24,25.1,24,24,0.0,0,0,0,0,0
5,quadratic-svr_1,23.6,23,23,23.6,23,23,0.0,0,0,0,0,0
6,cubic-svr_1,26.1,25,25,26.1,25,25,0.0,0,0,0,0,0


In [49]:
os.makedirs("plots", exist_ok=True)
for domain in IPC2023_LEARNING_DOMAINS:
  try:
    data = get_data(domain)

    max_val = 0

    for config in CONFIGS:
      cs = np.array(data[config]["cost"])
      c_opts = np.array(data[config]["c_opt"])
      index = np.where(cs != -1)
      cs = cs[index]
      c_opts = c_opts[index]
      plt.scatter(c_opts, cs, label="_".join(config))
      max_val = max([max_val, max(cs), max(c_opts)])
    
    plt.title(domain)
    plt.plot([1, max_val], [1, max_val], alpha=0.5, linestyle="dotted", c='k')
    plt.ylabel("C")
    plt.xlabel("C* (not necessarily optimal)")
    plt.legend()
    # plt.show()
    plt.savefig(f"plots/ipc2023-learning-{domain}-cost.png")
  except:
    pass
  plt.clf()
print("c vs c* saved")

c vs c* saved


<Figure size 640x480 with 0 Axes>

In [50]:
os.makedirs("plots", exist_ok=True)
for domain in IPC2023_LEARNING_DOMAINS:
  try:
    data = get_data(domain)

    max_val = 0

    my_dict = {}

    fig, ax = plt.subplots()

    for config in CONFIGS:
      expanded = np.array(data[config]["expanded"])
      # index = np.where(expanded != -1)
      expanded = expanded[index]
      my_dict["_".join(config)] = expanded
    
    plt.yscale("log")
    plt.title(domain)
    ax.boxplot(my_dict.values())
    ax.set_xticklabels(my_dict.keys())
    # plt.ylim(bottom=1)
    # plt.show()
    plt.savefig(f"plots/ipc2023-learning-{domain}-expanded.png")
  except:
    pass
  plt.clf()
print("expansions saved")

expansions saved


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>